In [ ]:
import os

import re

from bs4 import BeautifulSoup

import pandas as pd

In [159]:
odkud_brat = "downloads/aleph"

In [ ]:
ddd = pd.read_xml(os.path.join('downloads/cnb.xml','cnb.xml'))

In [160]:
stranky = [i for i in os.listdir(odkud_brat)]

In [161]:
def scrape_aleph(soubor):
    
    with open(os.path.join(odkud_brat, soubor), "r", encoding="utf-8") as stranka:
        stranka = stranka.read()
    
    soup = BeautifulSoup(stranka, "html.parser")
    
    kniha = {}
    kniha['A_isbn'] = soubor.split('.')[0]
    klice_pouzite = []
    
    for radek in soup.find_all("tr"):
        if len(radek.find_all('td')) == 2:
            
            klic_pracovni = radek.find('td').text.strip()
            
            if klic_pracovni != '':
                
                klic = klic_pracovni.replace(',','').replace('- ','').replace(' ','_').lower().strip()
            
            if radek.find_all('td')[1].find('a'):
                hodnota = [odkaz.text.replace('\xa0',' ').strip() for odkaz in radek.find_all('td')[1].find_all('a')]
            else:
                hodnota = radek.find_all('td')[1].text.replace('\xa0',' ').strip()
                
            if klic_pracovni in ['Forma, žánr','Forma, žánr-angl.']:
                hodnota = [h.strip() for h in hodnota.split('*') if len(h.strip()) > 0]
                                
            if klic_pracovni not in klice_pouzite:
                
                klice_pouzite.append(klic_pracovni)
                    
                klic = klic.replace(',','').replace('- ','').replace(' ','_').lower().strip()
                    
                kniha[f"""A_{klic}"""] = hodnota
                
            else:
                
                if isinstance(kniha[f"""A_{klic}"""], str):
                    kniha[f"""A_{klic}"""] = [kniha[f"""A_{klic}"""]]
                
                if isinstance(hodnota, str):
                    kniha[f"""A_{klic}"""].append(hodnota)
                elif isinstance(hodnota, list):
                    kniha[f"""A_{klic}"""] = kniha[f"""A_{klic}"""] + hodnota
    
                kniha[f"""A_{klic}"""] = [x for x in kniha[f"""A_{klic}"""] if len(x) > 0]
    
    return kniha

In [162]:
scrape_aleph("9788087938652.html")

{'A_isbn': '978-80-87938-65-2 (brožováno)',
 'A_nkc_úplné_zobrazení_záznamu': '',
 'A_záznam_________1_z_________1': '',
 'A_hlavní_záhlaví': ['Kopecký, Rudolf, 1893-1981'],
 'A_název': ['Rudolf Kopecký: vzpomínky starého novináře : "proti Benešovi, nacistům a komunistům" / Martin Nekola'],
 'A_vydání': 'Vydání první',
 'A_nakladatel': '[Beroun] : Machart, 2018',
 'A_popis_(rozsah)': '511 stran : ilustrace, portréty, faksimile ; 24 cm',
 'A_typ_obsahu': 'text',
 'A_typ_nosiče': '',
 'A_exempláře': [],
 'A_číslo_nár.bibl.': 'cnb003004373',
 'A_další_původce': ['Nekola, Martin, 1982-'],
 'A_poznámka': ['Anglické resumé',
  'Obsahuje bibliografické odkazy a rejstřík'],
 'A_předmět._heslo': ['Kopecký, Rudolf, 1893-1981',
  'novináři -- Česko -- 20. století',
  'svobodné zednářství',
  'politické názory a postoje',
  'exilová politická činnost',
  'exilová publicistika',
  'Československo -- politika a vláda'],
 'A_chronolog._údaj': ['* 20. století', '* 1893-1981'],
 'A_forma_žánr': ['autob

In [163]:
df = []
for s in stranky:
    df.append(scrape_aleph(s))
df = pd.DataFrame(df)
df

,A_isbn,A_nkc_úplné_zobrazení_záznamu,A_záznam_________1_z_________1,A_hlavní_záhlaví,A_název,A_vydání,A_nakladatel,A_popis_(rozsah),A_typ_obsahu,A_typ_nosiče,...,A_obsahuje_též,A_dg_request,A_další_korporace,A_obsahuje_(neúpl.),A_poslední_nakl.,A_roky_/_sv.,A_vydáváno,A_souč._periodicita,A_jiný_nosič,A_dříve_jako
0,978-3-903124-12-7 (vázáno),,,"[Cikán, Ondřej, 1985-]",[Nejsladší potrava : múza je čas všeho / Ondře...,1. vydání,"Vídeň ; Praha : Kētos, 2020",61 stran : ilustrace ; 19 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,978-3-903124-25-7 (vázáno),,,"[Bondy, Egon, 1930-2007]",[In Straßenbahnen : totaler Realismus und Pein...,1. Auflage,"Prag ; Wien : Kētos, 2023",253 stran ; 19 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,978-3-903124-30-1 (vázáno),,,"[Ciprysová, Iveta, 1993-]",[Auf der Speerspitze des Brunnens / Iveta Cipr...,I. Auflage,"Wien ; Prag ; Kētos, 2022",108 stran ; 19 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"978-80-00-05128-4 (vázáno) : Kč 229,00",,,"[Lamková, Hana, 1934-]","[Káťa a Škubánek zase spolu / Hana Lamková, Jo...",2. vydání,"V Praze : Albatros, 2018",67 stran : barevné ilustrace ; 27 cm,statický obraz,[svazek],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,978-80-00-06333-1 (vázáno) * 978-80-00-063331 ...,,,"[Peroutková, Ivana, 1960-]",[Anička ve městě / Ivana Peroutková ; ilustrov...,3. vydání,"V Praze : Albatros, 2021",116 stran : barevné ilustrace ; 22 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,"978-80-908723-0-1 (brožováno) : Kč 339,00",,,NaN,"[Mami, co je to čupák? : bláznivé příběhy o ro...",NaN,"[Praha] : Veritas, 2022",230 stran : barevné ilustrace ; 21 cm,text,,...,NaN,NaN,[Západočeská univerzita. Fakulta designu a umě...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,[978-80-908764-7-7 (brožováno)],,,"[Klukan, Petr, 1961-]",[Húlava nad buzerplacem / Petr Klukan],První vydání,"[Nová Říše] : Dobrý důvod, 2023",133 stran : ilustrace (převážně barevné) ; 18 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,[978-80-908764-8-4 (brožováno)],,,"[Tintěrová, Monika, 1991-]",[Starý paní mají v ústech porcelán / Monika Ti...,První vydání,"[Nová Říše] : Dobrý důvod, 2023",65 stran ; 18 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,978-80-908873-3-6 (brožováno),,,"[Dušková, Ruth]",[Skládačka života / Ruth Dušková],První vydání,"Praha : CMYK 57, s.r.o. - Vydej knihu, 2023",98 stran : ilustrace ; 16 cm,text,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
df.columns

Index(['A_isbn', 'A_nkc_úplné_zobrazení_záznamu',
       'A_záznam_________1_z_________1', 'A_hlavní_záhlaví', 'A_název',
       'A_vydání', 'A_nakladatel', 'A_popis_(rozsah)', 'A_typ_obsahu',
       'A_typ_nosiče', 'A_exempláře', 'A_číslo_nár.bibl.', 'A_poznámka',
       'A_forma_žánr', 'A_skupina_konspektu', 'A_mdt', 'A_forma_žánr-angl.',
       'A_skc', 'A_google', 'A_systém._číslo', 'A_edice', 'A_varianta_názvu',
       'A_název_originálu', 'A_další_původce', 'A_resumé', 'A_z_cyklu',
       'A_předmět._heslo', 'A_předm._hesla-angl.', 'A_chronolog._údaj',
       'A_anotace', 'A_nakl._údaje', 'A_předm._heslo-angl.', 'A_obsahuje',
       'A_související_jedn.', 'A_odkazy', 'A_copyright', 'A_obsahuje_též',
       'A_dg_request', 'A_další_korporace', 'A_obsahuje_(neúpl.)',
       'A_poslední_nakl.', 'A_roky_/_sv.', 'A_vydáváno', 'A_souč._periodicita',
       'A_jiný_nosič', 'A_dříve_jako'],
      dtype='object')

In [165]:
df['A_hlavní_záhlaví'].dropna().sample(60)

433                       [Nachtmanová, Petra, 1972-]
262                       [Richterová, Sylvie, 1945-]
297                                    [Kučera, Jiří]
214                                   [Tarina. Česky]
475                        [Landsman, Dominik, 1985-]
1                            [Bondy, Egon, 1930-2007]
118                             [Janů, Daniel, 1970-]
258                       [Červenka, Ladislav, 1946-]
208                                 [Gravensteen, A.]
71                           [Viewegh, Michal, 1962-]
147                               [Steinbauer, Boris]
459                   [Mácha, Karel Hynek, 1810-1836]
11                               [Kraus, Ivan, 1939-]
183                         [Šťastná, Barbora, 1973-]
353                               [Šíp, Karel, 1945-]
248                   [Pittnerová, Vlasta, 1858-1926]
437                             [Pleskanková, Silvie]
249                          [Řeháčková, Věra, 1950-]
267                         

In [166]:
df.to_json(os.path.join('data_raw','aleph_raw.json'))